In [ ]:
import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.preprocessing import OrdinalEncoder, normalize
from sklearn.model_selection import GroupKFold, StratifiedKFold
from sklearn.metrics import confusion_matrix
import torch
from torch.utils.data import DataLoader
import math
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.metrics import confusion_matrix
#from sklearn.metrics.classification import accuracy_score, log_loss
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,fbeta_score, matthews_corrcoef
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier


###Import the final CSV-file

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/T2D-ML/Datasets/Final_T2D_Meta_GSE81608_filtered_metadata.csv')

df.head()

,Sample_Names,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,...,MIR6830,TCF20.7,MIR548AH,MIR522,MIR6715B,MIR3116-2,MIR3202-2,HSFX3,MIR4481,condition
0,GSM2157899,4.0685,0.0,0.0000,6.0980,0.0000,0.0,0.0000,8.6118,0.0000,...,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,GSM2157900,0.8755,0.0,6.2501,10.3465,0.0000,0.0,0.0000,16.2721,85.7705,...,0.0,0.8168,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,GSM2157901,1.2943,0.0,62.6107,12.2002,1.8483,0.0,7.4244,4.1191,1.6251,...,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,GSM2157902,0.0000,0.0,0.0000,22.1665,0.0000,0.0,0.0000,0.0000,0.7967,...,0.0,10.8795,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,GSM2157903,11.4406,0.0,18.2915,6.2444,0.0000,0.0,0.0000,3.4575,0.0000,...,0.0,5.5639,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [ ]:
df.columns

Index(['Sample_Names', 'TSPAN6', 'TNMD', 'DPM1', 'SCYL3', 'C1orf112', 'FGR',
       'CFH', 'FUCA2', 'GCLC',
       ...
       'MIR6830', 'TCF20.7', 'MIR548AH', 'MIR522', 'MIR6715B', 'MIR3116-2',
       'MIR3202-2', 'HSFX3', 'MIR4481', 'condition'],
      dtype='object', length=28091)

In [ ]:
#df['churn'] = df['churn'].apply(np.int64)
y_true = df['condition']
df.drop(['Sample_Names','condition'], axis=1, inplace=True)
y_true = list(map(int, y_true.values))

In [ ]:
df.head()

,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,MIR3913-2,MIR6830,TCF20.7,MIR548AH,MIR522,MIR6715B,MIR3116-2,MIR3202-2,HSFX3,MIR4481
0,4.0685,0.0,0.0000,6.0980,0.0000,0.0,0.0000,8.6118,0.0000,35.6156,...,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.8755,0.0,6.2501,10.3465,0.0000,0.0,0.0000,16.2721,85.7705,0.0000,...,0.0,0.0,0.8168,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.2943,0.0,62.6107,12.2002,1.8483,0.0,7.4244,4.1191,1.6251,0.0000,...,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0000,0.0,0.0000,22.1665,0.0000,0.0,0.0000,0.0000,0.7967,19.5898,...,0.0,0.0,10.8795,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,11.4406,0.0,18.2915,6.2444,0.0000,0.0,0.0000,3.4575,0.0000,17.6535,...,0.0,0.0,5.5639,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#Confusion Matrix

In [ ]:
#For test and train data.
def plot_confusion_matrix(test_y, predict_y):
    C = confusion_matrix(test_y, predict_y)

    A = (((C.T)/(C.sum(axis=1))).T)
    B = (C/C.sum(axis=0))

    plt.figure(figsize = (20,4))
    labels = [1,2]

    cmap = sns.light_palette("blue")
    plt.subplot(1,3,1)
    sns.heatmap(C, annot = True, cmap = cmap, fmt = ".3f", xticklabels = labels, yticklabels = labels)
    plt.xlabel("Predicted Class")
    plt.ylabel("Original Class")
    plt.title("Confusion matrix")

    plt.subplot(1,3,2)
    sns.heatmap(B, annot = True, cmap = cmap, fmt = ".3f", xticklabels = labels, yticklabels = labels)
    plt.xlabel("Predicted Class")
    plt.ylabel("Original Class")
    plt.title("Precision matrix")

    plt.subplot(1, 3, 3)
    sns.heatmap(A, annot = True, cmap = cmap, fmt = ".3f", xticklabels = labels, yticklabels = labels)
    plt.xlabel("Predicted Class")
    plt.ylabel("Orziginal Class")
    plt.title("Recall matrix")

    plt.show()

def curve(a, rf_fpr, rf_tpr, rf_auc):
  # plt.plot(r_fpr, r_tpr, linestyle='--', label='Random prediction (AUROC = %0.3f)' % r_auc)
  plt.plot(rf_fpr, rf_tpr, marker='.', label= a + ' (AUROC = %0.3f)' % rf_auc)
  # Title
  plt.title('ROC plot without oversampling')
  # Axis labels
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  # Show legend
  plt.legend() # 
  # Show plot
  plt.show()



#All Scores with True Positive, True Negative

In [ ]:
def perf_measure(y_actual, y_hat, log_ls, r_auc,cl_names):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
           TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           FP += 1
        if y_actual[i]==y_hat[i]==0:
           TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
           FN += 1
    
    TPR = TP/ (TP + FN)
    FPR = FP / (FP + TN)
    accuracy = (TP + TN)/(TP + FP + TN + FN)
    precision = (TP)/(TP + FP)
    specificity = (TN)/(TN + FP)
    recall = (TP)/(TP + FN)
    g_mean = math.sqrt( recall*specificity)
    f1score = (2 * precision * recall) / (precision + recall)

    print("Accuracy: %.3f" %accuracy)
    print("Log_Loss: %.3f" %log_ls)
    print("Precision: %.3f" %precision)
    print("Recall: %.3f" %recall)
    print("RocAuc: %.3f" %r_auc)
    print("Specificity: %.3f" %specificity)
    print("F1_Score: %.3f" %f1score)
    print("G_Mean: %.3f" %g_mean)


    print("Total True Positive: ", TP)
    print("Total True Negative: ", TN)
    print("Total False Positive: ", FP)
    print("Total False Negative: ", FN)
    print("True Positive Rate: %.3f" %TPR)
    print("False Positive Rate: %.3f" %FPR)

    my_dict["Classifier_Names"].append(cl_names)
    my_dict["Acc"].append(accuracy)
    my_dict["Log_L"].append(log_ls)
    my_dict["Pre"].append(precision)
    my_dict["Rec"].append(recall)
    my_dict["R_Auc"].append(r_auc)
    my_dict["Speci"].append(specificity)
    my_dict["F1_S"].append(f1score)
    my_dict["G_Mean"].append(g_mean)
    my_dict["All_TP"].append(TP)
    my_dict["All_TN"].append(TN)
    my_dict["TPR"].append(TPR)
    my_dict["FPR"].append(FPR)
  

    #return(TP, FP, TN, FN)

#Classifier Models Function

In [ ]:
def train_test(X_train, X_test, y_train, y_test):
  bold_start = '\033[1m'
  bold_end   = '\033[0m'
  k = 0
  for i in range(50):
    if i == 0:
      classifier = RandomForestClassifier(n_estimators=200, random_state=1)
      a = 'Random Forest'

    elif i == 1:
      classifier = AdaBoostClassifier(n_estimators=200, random_state=1, learning_rate=0.01)
      a = 'AdaBoost'

    elif i == 2:
      classifier = GradientBoostingClassifier(n_estimators=200, random_state=1, learning_rate=0.01)
      a = 'Gardient Boosting'
    
    elif i == 3:
      classifier =LogisticRegression(class_weight= 'balanced')
      a = 'Logistic Regression'

    elif i == 4:
      classifier =DecisionTreeClassifier(class_weight= 'balanced')
      a = 'Decision Tree'

    else:
      classifier = XGBClassifier (n_estimators=200, random_state=1, learning_rate=0.01)
      a = 'XGBoost'
      k = 1
  
    classifier.fit(X_train,y_train)
    predict_y = classifier.predict_proba(X_test)
    predicted_y =np.argmax(predict_y,axis=1)

    print(bold_start, a, bold_end)
    #acc = accuracy_score(y_test, predicted_y)
    log_ls = log_loss(y_test, predict_y, labels=classifier.classes_, eps=1e-15)
    #print('Accuracy: %.3f' %(accuracy_score(y_test, predicted_y)))
    #print('Log Loss: %.3f' %(log_loss(y_test, predict_y, labels=classifier.classes_, eps=1e-15)))
    #print("Precision Score: ",precision_score(y_test, predicted_y, average='macro'))
    
    #print("Recall score: ",recall_score(y_test, predicted_y, average='macro'))
    r_auc = roc_auc_score(y_test, predicted_y)
    #print('AucRoc Score: ', r_auc)
    perf_measure(y_test, predicted_y, log_ls, r_auc,a)
    plot_confusion_matrix(y_test, predicted_y)
    r_fpr, r_tpr, _ = sklearn.metrics.roc_curve(y_test, predicted_y)
    curve(a,r_fpr, r_tpr, r_auc)
    
    if k == 1:
      break


#Run models with train test split

In [ ]:
#Spliting oversample data(Smote)

X_train, X_test, y_train, y_test = train_test_split(df, y_true, stratify = y_true, test_size = 0.2)
print("Number of data points in train data :", X_train.shape)
print("Number of data points in test data :", X_test.shape)

#Train and test oversample data (Smote)

my_dict = {'Classifier_Names': [], 'Acc': [], 'Log_L': [],  'Pre': [],  'Rec': [],  'R_Auc': [],  'Speci': [],  'F1_S': [],  'G_Mean': [], 'All_TP': [], 'All_TN' : [],  'TPR': [],  'FPR': []}  

train_test(X_train, X_test, y_train, y_test)